In [ ]:
import pickle as pkl
import time
import os
import numpy as np
import sys
import mido
from mido import MidiFile ,MetaMessage, Message, MidiTrack, second2tick
from models.LSTM_Minimized import create_network, sample_with_temp

import matplotlib.pyplot as plt


# parameters

In [ ]:
# run params
section = 'compose'
run_id = '1122'
music_name = 'midis'
run_folder = 'run/{}/'.format(section)
run_folder += '_'.join([run_id, music_name])

# model params
embed_size = 200
rnn_units = 256
use_attention = True


## load the lookup tables

In [ ]:

store_folder = os.path.join(run_folder, 'store')

with open(os.path.join(store_folder, 'distincts'), 'rb') as filepath:
    distincts = pkl.load(filepath)
    command_channel_names, n_command_channel, value1_names, n_value1, duration_names, n_durations = distincts

with open(os.path.join(store_folder, 'lookups'), 'rb') as filepath:
    lookups = pkl.load(filepath)
    command_channel_to_int, int_to_command_channel, value1_to_int, int_to_value1, duration_to_int, int_to_duration = lookups

## build the model

In [ ]:
weights_folder = os.path.join(run_folder, 'weights')
#weights_file = 'weights_custom.h5'
weights_file = 'weights_finished.h5'

model, att_model = create_network(n_command_channel, n_value1, n_durations, embed_size, rnn_units, use_attention)

# Load the weights to each node
weight_source = os.path.join(weights_folder,weights_file)
model.load_weights(weight_source)
model.summary()

## build your own phrase

In [ ]:
# prediction params
commands_temp = 0.5
values1_temp = 0.5
duration_temp = 0.5

max_extra_commands = 50
max_seq_len = 128
seq_len = 64

# notes = ['START', 'D3', 'D3', 'E3', 'D3', 'G3', 'F#3','D3', 'D3', 'E3', 'D3', 'G3', 'F#3','D3', 'D3', 'E3', 'D3', 'G3', 'F#3','D3', 'D3', 'E3', 'D3', 'G3', 'F#3']
# durations = [0, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2]


# notes = ['START', 'F#3', 'G#3', 'F#3', 'E3', 'F#3', 'G#3', 'F#3', 'E3', 'F#3', 'G#3', 'F#3', 'E3','F#3', 'G#3', 'F#3', 'E3', 'F#3', 'G#3', 'F#3', 'E3', 'F#3', 'G#3', 'F#3', 'E3']
# durations = [0, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2]

command_channel = ['START']
values1 = [-1]
durations = [0]

#if seq_len is not None:
#    notes = ['START'] * (seq_len - len(notes)) + notes
#    durations = [0] * (seq_len - len(durations)) + durations

sequence_length = 128

## Generate notes from the neural network based on a sequence of notes

In [ ]:
#xy = "note_on.1"
#command, channel = xy.split(".")
#print(command)
#print(channel)

In [ ]:
def generateMidoMessage(command_channel, channel, value1, duration):
    command, channel = command_channel.split('.')
    if command != 'time_signature' and command != 'key_signature':
        value1 = int(value1)
        value2 = int(64)
        
    duration = int(duration)
    pChannel = int(channel)
    
    if command == 'set_tempo':
        return MetaMessage(command, tempo=value1, time=duration)
    elif command == 'key_signature':    
        return MetaMessage(command, key=value1, time=duration)
    elif command == 'end_of_track':
        return MetaMessage(command, time=duration)
    elif command == 'note_on' or command == 'note_off':
        return Message(command, channel=pChannel, note=value1, velocity=value2, time=duration)
    elif command == 'control_change':
        return Message(command, channel=pChannel, control=value1,  value=value2, time=duration)
    elif command == 'program_change':
        return Message(command, channel=pChannel, program=value1, time=duration)
    elif command == 'pitchwheel':
        return Message(command, channel=pChannel, pitch=value1, time=duration)

In [ ]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

prediction_output = []

commands_input_sequence = []
value1_input_sequence = []
durations_input_sequence = []

overall_preds = []

for co, v1, d in zip(command_channel, values1, durations):
    command_int = command_channel_to_int[co]
    value1_int = value1_to_int[str(v1)]
    duration_int = duration_to_int[d]
    
    commands_input_sequence.append(command_int)
    value1_input_sequence.append(value1_int)
    durations_input_sequence.append(duration_int)
    
    prediction_output.append([co, v1, d])
    
    if co == 'START':
        createdMidi = MidiFile()
        track = MidiTrack()
        createdMidi.tracks.append(track)
    else:
        midi_note = note.Note(n)

        new_note = np.zeros(128)
        new_note[midi_note.pitch.midi] = 1
        overall_preds.append(new_note)


att_matrix = np.zeros(shape = (max_extra_commands+sequence_length, max_extra_commands))

for command_index in range(max_extra_commands):

    print(command_index)
    prediction_input = [
        np.asarray([commands_input_sequence])
        , np.asarray([value1_input_sequence])
        , np.asarray([durations_input_sequence])
       ]
    command_prediction, value1_prediction, durations_prediction = model.predict(prediction_input, verbose=0)
    if use_attention:
        att_prediction = att_model.predict(prediction_input, verbose=0)[0]
        att_matrix[(command_index-len(att_prediction)+sequence_length):(command_index+sequence_length), command_index] = att_prediction
            
    i1 = sample_with_temp(command_prediction[0], commands_temp)
    i3 = sample_with_temp(value1_prediction[0], values1_temp)
    i5 = sample_with_temp(durations_prediction[0], duration_temp)    

    command_result = int_to_command_channel[i1]
    value1_result = int_to_value1[i3]
    duration_result = int_to_duration[i5]
    
    print(command_result, value1_result, duration_result)
    
    prediction_output.append([command_result, value1_result, duration_result])

    commands_input_sequence.append(i1)
    value1_input_sequence.append(i3)
    durations_input_sequence.append(i5)
    
    if len(commands_input_sequence) > max_seq_len:
        commands_input_sequence = commands_input_sequence[1:]
        value1_input_sequence = value1_input_sequence[1:]
        durations_input_sequence = durations_input_sequence[1:]
        
    if command_result == 'START':
        break
        
print('Generated sequence of {} notes'.format(len(prediction_output)))

In [ ]:
prediction_output

## convert the output from the prediction to notes and create a midi file from the notes 

In [ ]:
output_folder = os.path.join(run_folder, 'output')

# create note and chord objects based on the values generated by the model
print(prediction_output)
for pattern in prediction_output:
    command_pattern, channel_pattern, value1_pattern, duration_pattern = pattern
    pTime = int(second2tick(duration_pattern, ticks_per_beat=default_ticks_per_second, tempo=default_tempo))
    # pattern is a chord
    if command_pattern == 'START':
        print('START')
        continue
    elif command_pattern == 'program_change':
        print('program_change')
        track.append(Message(command_pattern, channel=channel_pattern, program=int(value1_pattern), time=pTime))
        continue
    elif command_pattern == 'note_on' or command_pattern == 'note_off':
        print(command_pattern)
        track.append(Message(command_pattern, channel=channel_pattern, note=int(value1_pattern), time=pTime))
        continue

timestr = time.strftime("%Y%m%d-%H%M%S")
createdMidi.save(os.path.join(output_folder, 'output-minloss-' + timestr + '.mid'))